In [1]:
from sdRDM import DataModel
from sdRDM.generator import generate_python_api
import numpy as np
from datetime import datetime

In [2]:
# generate_python_api("specifications/new_calibration.md", "", "CaliPytion")

In [3]:
%reload_ext autoreload
%autoreload 2
from CaliPytion import Calibrator, Standard
from CaliPytion.modified import Sample

In [4]:
def random(xs):
    return xs+np.random.normal(0, 0.03, 1)


PARAM_A = 0.014

CONC = np.linspace(0, 200, 11)
ABSO = CONC*PARAM_A

In [5]:
standard = Standard(
    species_id="s0",
    name="ABTS",
    wavelength=340,
    ph=2,
    temperature=25,
    temperature_unit="C",
    created=datetime.now(),
)

# Add samples
for conc, abso in zip(CONC, ABSO):
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )

In [6]:
calibrator = Calibrator.from_standard(standard)

# Add models to calibrator
calibrator.add_to_models(name="linear", equation="a * concentration = signal")
calibrator.add_to_models(
    name="quadratic", equation="a * concentration**2 + b * concentration = signal")

print(calibrator)

Calibrator
├── id = calibrator0
├── standard
│   └── Standard
│       ├── id = standard1
│       ├── species_id = s0
│       ├── name = ABTS
│       ├── wavelength = 340.0
│       ├── samples
│       │   ├── 0
│       │   │   └── Sample
│       │   │       ├── id = sample0
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = -0.026808457319227978
│       │   ├── 1
│       │   │   └── Sample
│       │   │       ├── id = sample1
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = -0.006392601462152367
│       │   ├── 2
│       │   │   └── Sample
│       │   │       ├── id = sample2
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = 0.0262116366115173
│       │   ├── 3
│       │   │   └── Sample
│       │   │       ├── id = sample3
│       │   │       ├── concentration = 20.0
│     

In [7]:
model = calibrator.models[0]
model

Model(id='model1', name='linear', equation='a * concentration = signal', parameters=[], aic=None, bic=None, r2=None, residuals=[], rmsd=None)

In [8]:
res = model.fit_signals(
    concentrations=calibrator.concentrations,
    signals=calibrator.signals,
)

In [9]:
model._extract_parameters(res)

In [10]:
print(model)

Model
├── id = model1
├── name = linear
├── equation = a * concentration = signal
├── parameters
│   ├── 0
│   │   └── Parameter
│   │       ├── id = parameter0
│   │       ├── name = a
│   │       ├── value = 0.013970655802823254
│   │       ├── init_value = 0.1
│   │       ├── standard_error = 4.235014688809505e-05
│   │       ├── lower_bound = -inf
│   │       └── upper_bound = inf
│   └── 1
│       └── Parameter
│           ├── id = parameter1
│           ├── name = a
│           ├── value = 0.013970655802823254
│           ├── init_value = 0.1
│           ├── standard_error = 4.235014688809505e-05
│           ├── lower_bound = -inf
│           └── upper_bound = inf
├── aic = -233.1754512412218
├── bic = -231.67894367975532
├── r2 = 0.9989837208502537
├── residuals = [0.026808457319227978, 0.006392601462152367, -0.0262116366115173, 0.026221236173107676, 0.032471547271102325, ...]
└── rmsd = 0.028346140015175783

